# 朴素贝叶斯
朴素贝叶斯是基于贝叶斯定理与特征条件独立假设的分类方法。

1．朴素贝叶斯法是典型的生成学习方法。生成方法由训练数据学习联合概率分布 $P(X,Y)$，然后求得后验概率分布$P(Y|X)$。具体来说，利用训练数据学习$P(X|Y)$和$P(Y)$的估计，得到联合概率分布：

$$P(X,Y)＝P(Y)P(X|Y)$$
概率估计方法可以是极大似然估计或贝叶斯估计。

2．朴素贝叶斯法的基本假设是条件独立性，

$$\begin{aligned} P(X&amp;=x | Y=c_{k} )=P\left(X^{(1)}=x^{(1)}, \cdots, X^{(n)}=x^{(n)} | Y=c_{k}\right) \\ &amp;=\prod_{j=1}^{n} P\left(X^{(j)}=x^{(j)} | Y=c_{k}\right) \end{aligned}$$
这是一个较强的假设。由于这一假设，模型包含的条件概率的数量大为减少，朴素贝叶斯法的学习与预测大为简化。因而朴素贝叶斯法高效，且易于实现。其缺点是分类的性能不一定很高。

3．朴素贝叶斯法利用贝叶斯定理与学到的联合概率模型进行分类预测。

$$P(Y | X)=\frac{P(X, Y)}{P(X)}=\frac{P(Y) P(X | Y)}{\sum_{Y} P(Y) P(X | Y)}$$
将输入$x$分到后验概率最大的类$y$。

$$y=\arg \max _{c_{k}} P\left(Y=c_{k}\right) \prod_{j=1}^{n} P\left(X_{j}=x^{(j)} | Y=c_{k}\right)$$
后验概率最大等价于0-1损失函数时的期望风险最小化。

利用python实现朴素贝叶斯

In [1]:
# coding: utf-8
import math
import numpy as np
from sklearn.datasets import load_iris
from collections import Counter

from sklearn.model_selection import train_test_split


class NavieBayes(object):
    def __init__(self):
        self.model = None

    @staticmethod
    def mean(x):
        return sum(x) / len(x)

    def stdev(self, x):
        avg = self.mean(x)
        stdev = (math.sqrt(sum([(i - avg) ** 2 for i in x]))) / len(x)
        return stdev

    # 处理训练数据
    def summarize(self, X_train):
        result = [(self.mean(i), self.stdev(i)) for i in zip(*X_train)]
        return result

    # 计算概率密度函数
    def gaussian_probability(self, x, mean, stdev):
        exponent = math.exp(-(math.pow(x - mean, 2) /
                              (2 * math.pow(stdev, 2))))
        return (1 / (math.sqrt(2 * math.pi) * stdev)) * exponent

    def fit(self, X, y):
        labels = list(set(y))
        data = {label: [] for label in labels}
        for f, label in zip(X, y):
            data[label].append(f)
        self.model = {
            label: self.summarize(value)
            for label, value in data.items()
        }

    def calculate_probabilities(self, input_data):
        probabilities = {}
        for label, value in self.model.items():
            probabilities[label] = 1
            for i in range(len(value)):
                mean, stdev = value[i]
                probabilities[label] *= self.gaussian_probability(
                    input_data[i], mean, stdev)
        return probabilities

    def predict(self, X_test):
        label = sorted(
            self.calculate_probabilities(X_test).items(),
            key=lambda x: x[-1])[-1][0]
        return label

    def score(self, X_test, y_test):
        right = 0
        for X, y in zip(X_test, y_test):
            label = self.predict(X)
            if label == y:
                right += 1
        return right / float(len(X_test))

测试写好的模型

In [2]:
 data = load_iris()
X = data['data']
y = data['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=True, random_state=1)
clf = NavieBayes()
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

0.9736842105263158


利用sklearn封装好的来对比

In [3]:
from sklearn.naive_bayes import GaussianNB
clf_sklearn = GaussianNB()
clf_sklearn.fit(X_train, y_train)
clf_sklearn.score(X_test, y_test)

0.9736842105263158

代码参考自：https://github.com/fengdu78/lihang-code/